# Description
This notebook will help setup and run a HAPI FHIR server, and load synthetic data into it.

# Requirements
* Python with the `phdi` package installed
* Java 11+ JDK - not JRE (for Synthea)
* Docker (or equivalent)

# Setup
Creating a local test environment consists of a few steps outlined below. First, we must identify a running FHIR server to use as a test harness. The instructions below specify how to run a docker container with the HAPI FHIR server, generate data using Synthea, and load that data into the FHIR server. The method outlined here runs a transient FHIR server - anything loaded in to it will be destroyed when the container is stopped.

## Run Docker
In a separate terminal window, run the following command:
```
docker run -p 8080:8080 hapiproject/hapi:latest
```

## Generate Test Data using Synthea
The following cell will generate randomized FHIR resources using Synthea.
In order to control how the data is generated, feel free to modify the `synthea_command_arguments` to do what you need. Information about Synthea options is available [here](https://github.com/synthetichealth/synthea/wiki/Basic-Setup-and-Running#running-syntheatm).

In [ ]:
import os
import requests
import subprocess

######
# User-defined settings
######

# The following arguments will be appended to the command line string.
# More info available here: https://github.com/synthetichealth/synthea/wiki/Basic-Setup-and-Running#running-syntheatm
synthea_command_arguments = ["-p", "100"]



synthea_url = "https://github.com/synthetichealth/synthea/releases/download/master-branch-latest/synthea-with-dependencies.jar"

# Download the synthea jar file
synthea_jar_response = requests.get(synthea_url, stream=True)
if not os.path.isdir("target"):
    os.mkdir("target")
if not os.path.exists("target/synthea-with-dependencies.jar"):
    print("Downloading synthea-with-dependencies.jar...")
    with open("target/synthea-with-dependencies.jar","wb") as file:
        for chunk in synthea_jar_response.iter_content(chunk_size=1024):
            file.write(chunk)

# Create Synthea data
synthea_args = ["java", "-jar", "synthea-with-dependencies.jar"]
synthea_args.extend(synthea_command_arguments)
print(synthea_args)
subprocess.run(args=synthea_args, cwd="target")

## Load Data
The command above should have generated a set of syntentic patients in FHIR format.  Now we need to upload them to the FHIR server.

In [ ]:
import json
import phdi.fhir.transport
import os
import pathlib

base_path = os.path.join("target","output","fhir")
for fhir_filename in os.listdir(base_path):
    fhir_filepath = os.path.join(base_path, fhir_filename)
    print(f"Evaluating {fhir_filepath}")

    if os.path.isfile(fhir_filepath) and fhir_filepath.endswith(".json"):
        with open(fhir_filepath) as fhir_file:
            print(f"Importing {fhir_filepath}")
            phdi.fhir.transport.http.http_request_with_reauth(cred_manager=None, url="http://localhost:8080/fhir", retry_count=3, request_type="POST", allowed_methods="POST", headers={}, data=json.load(fhir_file))